## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("WeatherPy_Database.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mahebourg,-20.4081,57.7000,76.24,83,100,6.71,MU,2022-01-09 23:13:06
1,1,Hamilton,39.1834,-84.5333,35.64,62,0,3.00,US,2022-01-09 23:10:50
2,2,Kaduy,59.2000,37.1500,11.93,98,98,2.55,RU,2022-01-09 23:13:06
3,3,Upernavik,72.7868,-56.1549,0.36,98,100,11.56,GL,2022-01-09 23:13:07
4,4,Lata,40.1629,-8.3327,56.71,99,100,2.82,PT,2022-01-09 23:13:07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mahebourg,-20.4081,57.7000,76.24,83,100,6.71,MU,2022-01-09 23:13:06
9,9,Carnarvon,-24.8667,113.6333,73.47,60,0,13.80,AU,2022-01-09 23:13:08
13,13,Mataura,-46.1927,168.8643,66.47,57,68,1.41,NZ,2022-01-09 23:13:10
14,14,Rikitea,-23.1203,-134.9692,78.30,79,100,18.97,PF,2022-01-09 23:13:10
17,17,Kieta,-6.2167,155.6333,84.94,71,28,10.00,PG,2022-01-09 23:13:11
18,18,Dingle,10.9995,122.6711,74.48,93,71,5.64,PH,2022-01-09 23:13:11
22,22,Manzini,-26.2500,31.5000,67.24,90,71,0.18,SZ,2022-01-09 23:13:12
23,23,Hermanus,-34.4187,19.2345,71.80,86,100,11.27,ZA,2022-01-09 23:13:12
24,24,Pisco,-13.7000,-76.2167,73.45,83,0,21.85,PE,2022-01-09 23:13:12
26,26,Sinnamary,5.3833,-52.9500,76.12,82,98,7.36,GF,2022-01-09 23:13:13


In [5]:
# 4a. Determine if there are any empty rows.

#   determine if the preferred_cities_df DataFrame has any null values for any of the rows.

preferred_cities_df.count()

# or use any of the other two below

# preferred_cities_df.isnull().sum()
# preferred_cities_df.notnull().sum()

City_ID       290
City          290
Lat           290
Lng           290
Max Temp      290
Humidity      290
Cloudiness    290
Wind Speed    290
Country       288
Date          290
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. 

# the are not any empty row or null values according to results above

clean_df = preferred_cities_df

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mahebourg,MU,76.24,-20.4081,57.7000,
9,Carnarvon,AU,73.47,-24.8667,113.6333,
13,Mataura,NZ,66.47,-46.1927,168.8643,
14,Rikitea,PF,78.30,-23.1203,-134.9692,
17,Kieta,PG,84.94,-6.2167,155.6333,
18,Dingle,PH,74.48,10.9995,122.6711,
22,Manzini,SZ,67.24,-26.2500,31.5000,
23,Hermanus,ZA,71.80,-34.4187,19.2345,
24,Pisco,PE,73.45,-13.7000,-76.2167,
26,Sinnamary,GF,76.12,5.3833,-52.9500,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df

clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mahebourg,MU,76.24,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Carnarvon,AU,73.47,-24.8667,113.6333,Hospitality Carnarvon
13,Mataura,NZ,66.47,-46.1927,168.8643,Ellie's Villa
14,Rikitea,PF,78.30,-23.1203,-134.9692,People ThankYou
17,Kieta,PG,84.94,-6.2167,155.6333,Uruna Bay Retreat


In [19]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 



locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))